# Legal NLP Results & Performance Visualizations

This notebook creates publication-quality visualizations for:

- Model performance metrics and evaluation results
- SHAP analysis and explainability insights  
- Clause extraction accuracy and confidence distributions
- Comparative analysis of different explainability methods
- Training progress and convergence plots

**Output**: High-resolution figures for academic presentations and papers.

In [ ]:
# Core libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json
from pathlib import Path
import sys

# Add project paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuration
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.labelsize': 14
})

# Create output directory
output_dir = project_root / 'visualizations' / 'figures'
output_dir.mkdir(parents=True, exist_ok=True)

# Load project data paths
data_dir = project_root / 'data'
models_dir = project_root / 'models'
results_dir = project_root / 'results'

print("✓ Results visualization environment configured")
print(f"📁 Output directory: {output_dir}")